#### Import packages

In [1]:
import requests
import urllib
import pandas as pd
import os
from bs4 import BeautifulSoup
import re

#### Download HTML pages

In [ ]:
def saveashtml(s,file_name):
    with open(file_name,'w') as f:
        f.write(s)
        f.close()

In [84]:
headers = {'User-Agent': 'whatever'}

start_year = 1950
end_year = 1995
url = "http://www.liiofindia.org/in/cases/cen/INSC/"
path = "Web Pages/"

for curr_year in range(start_year,end_year+1,1):
    flag = True
    i = 1
    while flag :
        url_new = url + str(curr_year) + "/" + str(i) + ".html"
        file_name = path + str(curr_year) + "_" + str(i) + ".html"
        result = requests.get(url_new,headers = headers)
        if result.status_code == int(404):
            flag = False
            i = 1
        else:
            saveashtml(result.text,file_name)
            i = i+1

#### Data Scraping from HTML pages

In [2]:
def extract2df(soup,filename):
    head_str = ""
    civil_str = ""
    h = 0
    p_list = soup.find_all('p')
    for elem in p_list:
        if elem.text.strip().lower().startswith("HEADNOTE:".lower()):
            h = 1
            continue
        if elem.text.strip().lower().replace("  "," ").startswith("Civil Appell".lower()):
            h = 2
        if h == 1 :
            head_str = head_str + " ".join(elem.text.split())
        if h == 2 :
            civil_str = civil_str + " ".join(elem.text.split())
    if h == 2:
#         res = filename.replace(".html","").split("_")
#         year = int(res[0])
#         doc_no = int(res[1])
        df_obj = pd.DataFrame({"Summary":[head_str],"Full":[civil_str]})
#         df_obj = pd.DataFrame({"Summary":[head_str],"Full":[civil_str],"Year":[year],"Doc_no":[doc_no]})
    else:
        return [] 
    return df_obj

In [4]:
df = pd.DataFrame()

directory = "/Users/kowshikmoyya/Documents/PGDBA/IIT Semester/Information Retrieval/Project/Web Pages/"
for filename in os.listdir(directory):
    if filename.endswith(".html"):
        fname = os.path.join(directory,filename)
        f = open(fname)
        soup = BeautifulSoup(f,'html.parser')
        df_obj = extract2df(soup,filename)
        if len(df_obj) == 0:
            f.close()
            continue
        df = df.append(df_obj)
        f.close()
df_train = df.reset_index().drop(["index"],axis =1)
df_train.head()

,Summary,Full
0,When a tenant has neither paid nor tendered th...,CIVIL APPELLATE JURISDICTION: Civil Appeal No....
1,The appellant company used to employ Messrs. S...,"CIVIL APPELLATE, JURISDICTION: Civil Appeal No..."
2,By a writ petition under Article 226 of the Co...,CIVIL APPELLATE JURISDICTION: Civil Appeal No....
3,The respondent's 'father made a deposit of Rs....,CIVIL APPELLATE JURISDICTION : Civil Appeal No...
4,The appellant is the Mahant of Emhar Math of P...,CIVIL APPELLATE JURISDICTION: Civil Appeal No....


In [6]:
df_train.to_csv("test/data_scrap1.csv",index = False)

In [4]:
# df_train_new = df_train.sort_values(by= ["Year","Doc_no"],ascending = [True,True]).reset_index().drop(["index"],axis = 1)

In [8]:
# df_train_new.to_csv("/Users/kowshikmoyya/Documents/PGDBA/IIT Semester/Information Retrieval/Project/test/data_scrap.csv",index = False)